In [2]:
import geohash
import pandas as pd #数据分析
import numpy as np #科学计算
from pandas import Series,DataFrame
from datetime import datetime
import matplotlib.pyplot as plt
import csv
"""=====================================================================================================================
1 读取数据
"""
path ="E:/data2/"

# train_operation = pd.read_csv(path + "train_operation_tag.csv")
# test_operation = pd.read_csv(path + "test_operation_tag.csv")
test_transaction=pd.read_csv(path + "transaction_round1_new.csv")
data_tag=pd.read_csv(path + "tag_train_new.csv")

data_transaction=pd.read_csv(path + "transaction_train_new.csv")
train_transaction = pd.merge(data_transaction, data_tag, how='left', left_on='UID',right_on='UID')
temp1=train_transaction.isnull()
num=(temp1 == True).astype(bool).sum(axis=1)

is_null=DataFrame(list(zip(num)))
is_null=is_null.rename(columns={0:"is_null_num"})

train_transaction = pd.merge(train_transaction,is_null,left_index = True, right_index = True, how = 'outer')
"""【测试集】"""
temp2=test_transaction.isnull()
num=(temp2 == True).astype(bool).sum(axis=1)
is_null=DataFrame(list(zip(num)))
is_null=is_null.rename(columns={0:"is_null_num"})
test_transaction = pd.merge(test_transaction,is_null,left_index = True, right_index = True, how = 'outer')

"""=====================================================================================================================
3 删除列'code1','code2','acc_id1','acc_id2','acc_id3','merchant','ip1_sub','device1','device2'
"""
"""【训练集】"""
train_transaction.drop(['code1','code2','acc_id1','acc_id2','merchant','acc_id3','ip1_sub','device1','device2'],axis=1,inplace=True)

"""【测试集】"""
test_transaction.drop(['code1','code2','acc_id1','acc_id2','merchant','acc_id3','ip1_sub','device1','device2'],axis=1,inplace=True)
"""=====================================================================================================================
4设备标记 安卓=1 iPhone=0
"""
"""【训练集】"""
phone1=DataFrame(train_transaction['device_code3']).isnull()
phone_size=(phone1 == True).astype(bool).sum(axis=1)
phone2=DataFrame(list(zip(phone_size)))
iphone_android=phone2.rename(columns={0:"phone"})
train_transaction['device_code3'] =iphone_android
train_transaction.drop(['device_code1','device_code2'],axis=1,inplace=True)

"""【测试集】"""
phone3=DataFrame(test_transaction['device_code3']).isnull()
phone_size=(phone3 == True).astype(bool).sum(axis=1)
phone4=DataFrame(list(zip(phone_size)))
iphone_android=phone4.rename(columns={0:"phone"})
test_transaction['device_code3'] =iphone_android
test_transaction.drop(['device_code1','device_code2'],axis=1,inplace=True)

"""=====================================================================================================================
4 time时间归一化 小时
"""
"""【训练集】"""
train_transaction['time'] = pd.to_datetime(train_transaction['time'])
time_now = train_transaction['time'].apply(lambda x:int((x-datetime(2018,10,30,0,0,0)).seconds/3600))
train_transaction['time']= time_now

"""【测试集】"""
test_transaction['time'] = pd.to_datetime(test_transaction['time'])
time_now = test_transaction['time'].apply(lambda x:int((x-datetime(2018,10,30,0,0,0)).seconds/3600))
test_transaction['time']= time_now
"""=====================================================================================================================
5 对钱归一化  "trans_amt"本身已经脱敏
"""

"""=====================================================================================================================
6 trans_type1 交易类型1的处理
"""

n1=set(train_transaction['trans_type1'])
n2=set(test_transaction['trans_type1'])
n=n1.union(n2)
dic={}
for i,j in enumerate(n):
    dic[j]=i
"""【训练集】"""
# train_transaction['trans_type1']=train_transaction['trans_type1'].fillna(100)
train_transaction['trans_type1'] = train_transaction['trans_type1'].map(dic)

"""【测试集】"""
# test_transaction['trans_type1']=test_transaction['trans_type1'].fillna(100)
test_transaction['trans_type1'] = test_transaction['trans_type1'].map(dic)

"""=====================================================================================================================
7 trans_type2 交易类型2的处理
"""
"""【训练集】"""
train_transaction['trans_type2'] = train_transaction['trans_type2'].fillna(100)##用100 填充trans_type2的缺失值
train_transaction['trans_type2'] = train_transaction.trans_type2.apply(lambda x:float(x))

"""【测试集】"""
test_transaction['trans_type2'] = test_transaction['trans_type2'].fillna(100)##用100 填充trans_type2的缺失值
test_transaction['trans_type2'] = test_transaction['trans_type2'].apply(lambda x:float(x))

"""=====================================================================================================================
8 资金类型"amt_src1"的处理
"""
n1=set(train_transaction['amt_src1'])
n2=set(test_transaction['amt_src1'])
n=n1.union(n2)
dic={}
for i,j in enumerate(n):
    dic[j]=i
"""【训练集】"""
train_transaction['amt_src1'] = train_transaction['amt_src1'].map(dic)
"""【测试集】"""
test_transaction['amt_src1'] = test_transaction['amt_src1'].map(dic)

"""=====================================================================================================================
9 资金类型"amt_src2"的处理
"""
n1=set(train_transaction['amt_src2'])
n2=set(test_transaction['amt_src2'])
n=n1.union(n2)
dic={}
for i,j in enumerate(n):
    dic[j]=i

"""【训练集】"""
train_transaction['amt_src2'] = train_transaction['amt_src2'].map(dic)
"""【测试集】"""
test_transaction['amt_src2'] = test_transaction['amt_src2'].map(dic)
"""=====================================================================================================================
10 MAC地址的处理
"""
"""【训练集】"""
mac1 = DataFrame(train_transaction['mac1']).isnull()
mac_null=(mac1 == True).astype(bool).sum(axis=1)
mac2 = DataFrame(list(zip(mac_null)))
mac_is_null = mac2.rename(columns={0:"mac_is_null"})
train_transaction = train_transaction.join(mac_is_null)
train_transaction.drop(['mac1'],axis=1,inplace=True)

"""【测试集】"""
mac1 = DataFrame(test_transaction['mac1']).isnull()
mac_null=(mac1 == True).astype(bool).sum(axis=1)
mac2 = DataFrame(list(zip(mac_null)))
mac_is_null = mac2.rename(columns={0:"mac_is_null"})
test_transaction = test_transaction.join(mac_is_null)
test_transaction.drop(['mac1'],axis=1,inplace=True)
"""=====================================================================================================================
11 ip地址的处理
"""
"""【训练集】"""
ip=DataFrame(train_transaction['ip1']).isnull()
ip_null=(ip == True).astype(bool).sum(axis=1)
ip2=DataFrame(list(zip(ip_null)))
ip_is_null=ip2.rename(columns={0:"ip_is_null"})
train_transaction=train_transaction.join(ip_is_null)
train_transaction.drop(['ip1'],axis=1,inplace=True)

"""【测试集】"""
ip=DataFrame(test_transaction['ip1']).isnull()
ip_null=(ip == True).astype(bool).sum(axis=1)
ip2=DataFrame(list(zip(ip_null)))
ip_is_null=ip2.rename(columns={0:"ip_is_null"})
test_transaction=test_transaction.join(ip_is_null)
test_transaction.drop(['ip1'],axis=1,inplace=True)

"""=====================================================================================================================
12 地理位址"geocode"的处理
"""
"""【训练集】"""
train_transaction["geo_code"]= train_transaction["geo_code"].apply(lambda x: str(x))
train_transaction["geo_code"]= train_transaction["geo_code"].apply(lambda x: geohash.decode(x) if x != 'nan' else (200,200))
train_transaction["geo_code"]= train_transaction["geo_code"].apply(lambda x: list(x))

def address(x):
    w=200
    j=200
    if int(x[0])<200:
        #0-44/5=8
        w=int((int(x[0])-9)/5)
        #0-58/6=9
        if int(x[1])<70: j=10
        else: j=int((int(x[1])-76)/6)
    return str(w)+str(j)
train_transaction["geo_code"]= train_transaction["geo_code"].apply(lambda x: address(x))




"""【测试集】"""
test_transaction["geo_code"]= test_transaction["geo_code"].apply(lambda x: str(x))
test_transaction["geo_code"]= test_transaction["geo_code"].apply(lambda x: geohash.decode(x) if x != 'nan' else (200,200))
test_transaction["geo_code"]= test_transaction["geo_code"].apply(lambda x: list(x))

def address(x):
    w=200
    j=200
    if int(x[0])<200:
        #0-44/5=8
        w=int((int(x[0])-9)/5)
        #0-58/6=9
        if int(x[1])<70: j=10
        else: j=int((int(x[1])-76)/6)
    return str(w)+str(j)
test_transaction["geo_code"]= test_transaction["geo_code"].apply(lambda x: address(x))

"""合并"""

n1=set(train_transaction["geo_code"])
n2=set(test_transaction["geo_code"])
n=list(n1.union(n2))
dic={}
for i,j in enumerate(n):
    dic[j]=i

train_transaction["geo_code"]= train_transaction["geo_code"].apply(lambda x: str(x))
train_transaction["geo_code"]= train_transaction["geo_code"].map(dic)

test_transaction["geo_code"]= test_transaction["geo_code"].apply(lambda x: str(x))
test_transaction["geo_code"]= test_transaction["geo_code"].map(dic)

"""=====================================================================================================================
13 "market_code"和"market_type"的处理
    因为确实值太多，计算是否为空,删除market_code，保留market_type,并把空置为0
"""
"""【训练集】"""
temp2=DataFrame(train_transaction['market_type'].fillna(0.0))
temp2=temp2.rename(columns={'market_type':'market_code_type'})
train_transaction=train_transaction.join(temp2)
train_transaction.drop(['market_type','market_code'],axis=1,inplace=True)

"""【测试集】"""
temp3=DataFrame(test_transaction['market_type'].fillna(0.0))
temp3=temp3.rename(columns={'market_type':'market_code_type'})
test_transaction=test_transaction.join(temp3)
test_transaction.drop(['market_type','market_code'],axis=1,inplace=True)

"""
16 保存数据
"""

"""【训练集】"""
train_transaction.to_csv(path + 'train_transaction_V1.csv',index=False)

"""【测试集】"""
test_transaction.to_csv(path + 'test_transaction_V1.csv',index=False)

C:\Users\Simon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
